# GPT from scratch

In [20]:
with open('data/tiny_shakespeare.txt', 'r') as f:
    text = f.read()

In [21]:
print(len(text))

1115394


In [22]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [23]:
# create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decode: take a list of integers and output a string

print(encode("tomas"))
print(decode(encode("tomas")))


[58, 53, 51, 39, 57]
tomas


In [24]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:1000])

torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57

In [25]:
n = int(0.9*len(data)) # first 90% will be train, the rest validation
train_data = data[:n]
val_data = data[n:]

In [26]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [27]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} the target is {target}")

When input is tensor([18]) the target is 47
When input is tensor([18, 47]) the target is 56
When input is tensor([18, 47, 56]) the target is 57
When input is tensor([18, 47, 56, 57]) the target is 58
When input is tensor([18, 47, 56, 57, 58]) the target is 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [28]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel? 
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generates a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')

xb

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

In [29]:
yb

tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])

In [30]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        #idx and targets are both (B, T) tensor integers
        logits = self.token_embedding_table(idx) # B, T, C

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current contex
        for _ in range(max_new_tokens):

            # get the predictions
            logits, loss = self(idx)

            # each position logits[b, t, c]  represents the score for predicting the next token c in the sequence up to position t

            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B,C)

            #apply softmax to get probabilities
            # probs[b, c] = probability that the next token is c for the batch b
            probs = F.softmax(logits, dim=-1) # (B, C)

            # sample from the distribution
            # not always choose the most probable
            # result: one token for each element of batch
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

            # append sampled index to the running sequence
            # will be used in the next iteration
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        
        return idx


m = BigramLanguageModel(vocab_size) # vocab_size = len(chars)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [31]:
frase = "First Citizen, before we proceed any further..."
context = torch.tensor([encode(frase)], dtype=torch.long)
print(context.shape)  # torch.Size([1, 45])

torch.Size([1, 47])


In [32]:
logits

tensor([[-1.5101, -0.0948,  1.0927,  ..., -0.6126, -0.6597,  0.7624],
        [ 0.3323, -0.0872, -0.7470,  ..., -0.6716, -0.9572, -0.9594],
        [ 0.2475, -0.6349, -1.2909,  ...,  1.3064, -0.2256, -1.8305],
        ...,
        [-2.1910, -0.7574,  1.9656,  ..., -0.3580,  0.8585, -0.6161],
        [ 0.5978, -0.0514, -0.0646,  ..., -1.4649, -2.0555,  1.8275],
        [-0.6787,  0.8662, -1.6433,  ...,  2.3671, -0.7775, -0.2586]],
       grad_fn=<ViewBackward0>)

In [33]:
context = torch.tensor([encode("First")], dtype=torch.long)  # [18, 47, 56, 57, 58]
generated = m.generate(context, max_new_tokens=100)
print(generated)

tensor([[18, 47, 56, 57, 58,  7, 23, 21, 41, 24, 32, 11, 13, 41, 17, 24, 25, 53,
         32, 40, 60, 38, 60,  1, 15, 12, 52, 55,  7, 29, 17,  9,  9, 10, 15, 22,
         55, 49, 27, 23, 20,  7, 55, 11, 10, 50, 39,  2, 53, 47, 63, 61, 49, 20,
         48, 45, 15, 46, 64, 40, 29, 12, 59,  2,  9, 40, 24, 21, 45, 61, 43, 60,
         51, 63, 18, 22, 19, 33, 19, 54,  0, 61, 52, 37, 35, 51, 52, 62, 23, 35,
         35, 43, 60,  7, 58, 16, 55, 36, 17, 56, 34, 23, 24, 45, 22]])


In [34]:
print(decode(generated[0].tolist()))

First-KIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


The results are trash basically because the model is pure randomic. Lets train the model now

In [35]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [36]:
batch_size=32
for steps in range(10000):

    # sample random batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.382369041442871


In [37]:
context = torch.tensor([encode("First")], dtype=torch.long)  # [18, 47, 56, 57, 58]
generated = m.generate(context, max_new_tokens=100)
print(decode(generated[0].tolist()))

Firsthin br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulsee


### The mathematical trick of self-attention

In [38]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [39]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [40]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [41]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

note that x[0][j] is the mean of x[0][0] to x[0][j]

We can use matmul using this trick:

In [42]:
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10, (3,2)).float()
c = a @ b

In [43]:
print('A=' ,a)
print('\n')
print('B=' ,b)
print('\n')
print('C=' ,c)
print('\n')

A= tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])


B= tensor([[8., 6.],
        [5., 2.],
        [4., 4.]])


C= tensor([[8.0000, 6.0000],
        [6.5000, 4.0000],
        [5.6667, 4.0000]])




In [44]:
# version 2
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
print(wei)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) --> (B, T, C)


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


In [45]:
# version 3: use softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose (xbow, xbow3)

True

In [ ]:
# version 4: self-attention

# every single token at each position will emit 2 vectors,
# - query vector Q: this represents what am I looking for.
# - key vector K: this represents what do I contain.
# - value vector V: what information will I communicate if I'm relevant?

# The way we get affinities now in a sequence is doing a dot product between the keys and the queries.


torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B,T,C)

# single head perform attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)

trill = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [ ]:
# before wei was just a constant and it was applied in the same way for all the batch elements
# but now every single batch element will have different wei because every single batch element contains different tokens
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

without **value** (just using Q/K) I'll be restricted to aggregating the original token embeddings based on attention weights. The model can only decide "how much" to pay attention but not "what aspect" of the information to extract

With Value we allow:
- feature extraction
- information filtering
- richer representations


Notes:
- Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating

- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.

- Each example across batch dimension is not of course processed completely independently and never "talk" to each other

- In an "encoder" attention block juste delete the single line that does the masking with tril, allowing the tokens to communicate. here is called "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like in language modeling

- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention". "cross-attention" is used when there is a seperate source of nodes we like to pull information node into our nodes, keys and values separate from querys.

- Scaled attention additional divides wei by 1/sqrt(head_size). This makess it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Otherwise softmax will converge into one-hot vectors